### Wrangle the questions so that they can be ingested into the web application in certain format. 

In [1]:
# --- Setup
from __future__ import annotations

from pathlib import Path
from typing import Tuple, List, Optional

import pandas as pd

# Your robust MCQ parser
from src.general import extract_mcq_components

In [2]:

# --- File paths (edit if needed)
baseline_input = "./some_recorded_output/baseline_questions_cleaned.xlsx"
requesta_input = "./some_recorded_output/requesta_mcqs_cleaned.xlsx"
attention_input = "./some_recorded_output/attention_checker.xlsx"

baseline_output = "./some_recorded_output/baseline_questions_cleaned_parsed.xlsx"
requesta_output = "./some_recorded_output/requesta_mcqs_cleaned_parsed.xlsx"
attention_output = "./some_recorded_output/attention_checker_parsed.xlsx"

In [3]:

# --- Utilities
def _safe_extract(q: object) -> Tuple[str, List[Optional[str]]]:
    """Wrap extract_mcq_components with a safety net so a bad row never breaks the run."""
    try:
        stem, opts = extract_mcq_components(q)
        # Normalize empty strings to None for options
        norm_opts = [o if (o is not None and str(o).strip() != "") else None for o in (opts or [None]*4)]
        # Ensure exactly 4 options (A–D)
        if len(norm_opts) < 4:
            norm_opts = norm_opts + [None] * (4 - len(norm_opts))
        elif len(norm_opts) > 4:
            norm_opts = norm_opts[:4]
        return stem, norm_opts
    except Exception as e:
        # You may want to log/print e for debugging
        return "", [None, None, None, None]


def add_parsed_columns(df: pd.DataFrame, question_col: str) -> pd.DataFrame:
    """
    Add parsed MCQ columns to df:
      - question_item (stem)
      - optionA, optionB, optionC, optionD
    """
    if question_col not in df.columns:
        raise ValueError(f"Column '{question_col}' not found in DataFrame.")

    # Apply parsing
    parsed = df[question_col].apply(_safe_extract)

    # Split into stem + options
    stems = parsed.map(lambda t: t[0])
    options = parsed.map(lambda t: t[1])

    opt_df = pd.DataFrame(
        options.tolist(),
        columns=["optionA", "optionB", "optionC", "optionD"],
        index=df.index,
    )

    # Assemble the result
    out = df.copy()
    out["question_item"] = stems
    out[["optionA", "optionB", "optionC", "optionD"]] = opt_df

    return out


def _ensure_parent_dir(path_str: str | Path) -> None:
    p = Path(path_str)
    p.parent.mkdir(parents=True, exist_ok=True)


# --- 1) Read the three xlsx files
read_kwargs = dict(engine="openpyxl")  # explicit engine for reliability

baseline_df = pd.read_excel(baseline_input, **read_kwargs)
requesta_df = pd.read_excel(requesta_input, **read_kwargs)
attention_df = pd.read_excel(attention_input, **read_kwargs)

# --- 2) Parse stems & options from the three question columns
baseline_parsed = add_parsed_columns(baseline_df, "baseline_question")
requesta_parsed = add_parsed_columns(requesta_df, "requesta_question")
attention_parsed = add_parsed_columns(attention_df, "attention_question")

# --- 3) (already added) columns: question_item, optionA–optionD in each DataFrame

# --- 4) Write outputs
_ensure_parent_dir(baseline_output)
_ensure_parent_dir(requesta_output)
_ensure_parent_dir(attention_output)

baseline_parsed.to_excel(baseline_output, index=False, engine="openpyxl")
requesta_parsed.to_excel(requesta_output, index=False, engine="openpyxl")
attention_parsed.to_excel(attention_output, index=False, engine="openpyxl")

print("Done ✅")
print(f"Baseline → {baseline_output}")
print(f"ReQUESTA → {requesta_output}")
print(f"Attention → {attention_output}")


Done ✅
Baseline → ./some_recorded_output/baseline_questions_cleaned_parsed.xlsx
ReQUESTA → ./some_recorded_output/requesta_mcqs_cleaned_parsed.xlsx
Attention → ./some_recorded_output/attention_checker_parsed.xlsx


### Extract data from the xlsx files and ingest them into dictionaries for web application use

### First, Passage dictionary store all the 20 passages with their IDs

In [4]:
from __future__ import annotations

from pathlib import Path
from typing import Dict, List, Tuple
import re
import pandas as pd

In [5]:
INPUT_XLSX = Path("./some_recorded_output/baseline_questions_cleaned_parsed.xlsx")

OUTPUT_TXT = Path("./some_recorded_output/all_passages_dict.txt")

In [6]:
# Paragraph split on blank lines (one or more)
RE_BLANKLINE = re.compile(r"\n\s*\n", flags=re.UNICODE)


def normalize_paragraphs(text: str) -> str:
    """Ensure paragraphs are separated by exactly one '\n\n' and no trailing blank line.
    - Keep the original paragraph order
    - Normalize CRLF/CR to LF
    - Convert non-breaking spaces to regular spaces
    """
    if not isinstance(text, str):
        text = "" if text is None else str(text)

    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = text.replace("\u00A0", " ")  # NBSP -> space

    parts = [p.strip() for p in RE_BLANKLINE.split(text.strip()) if p.strip()]
    return "\n\n".join(parts)



def load_unique_texts(xlsx_path: Path, n: int = 20) -> List[Tuple[str, str]]:
    # .xlsx is Unicode-aware; no encoding arg needed here
    df = pd.read_excel(xlsx_path, engine="openpyxl", dtype={"textID": str})

    required_cols = {"textID", "text"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(
            f"Missing required columns in {xlsx_path.name}: {sorted(missing)}"
        )

    df = df[["textID", "text"]].dropna(subset=["textID", "text"]).copy()

    # De-duplicate by textID (keep first occurrence, preserve order), then take first n
    df = df.loc[~df["textID"].duplicated(keep="first")].head(n)

    pairs: List[Tuple[str, str]] = []
    for _, row in df.iterrows():
        tid = str(row["textID"]).strip()
        txt = normalize_paragraphs(row["text"])
        pairs.append((tid, txt))
    return pairs


def build_passages(pairs: List[Tuple[str, str]]) -> Dict[str, Dict[str, str]]:
    passages: Dict[str, Dict[str, str]] = {}
    for idx, (tid, txt) in enumerate(pairs, start=1):
        key = f"p{idx}"
        passages[key] = {
            "id": tid,
            "title": "Passage",
            "text": txt,
        }
        
    return passages


def _escape_triple_quotes(s: str) -> str:
    return s.replace('"""', '\\"""')


def _escape_single_quotes(s: str) -> str:
    return s.replace("\\", "\\\\").replace("'", "\\'")


def write_txt(passages: Dict[str, Dict[str, str]], out_path: Path) -> None:
    """Write a Python-friendly dict literal to a .txt file, using triple-quoted
    multi-line strings for the passage texts. File is saved as UTF-8.
    """
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with out_path.open("w", encoding="utf-8") as f:
        f.write("PASSAGES = {\n")
        for key, item in passages.items():
            f.write(f"    '{key}': {{\n")
            f.write(f"        'id': '{_escape_single_quotes(item['id'])}',\n")
            f.write("        'title': 'Passage',\n")
            f.write("        'text': \"\"\"\n")
            f.write(_escape_triple_quotes(item["text"]))
            f.write("\n\"\"\"\n")
            f.write("    },\n")
        f.write("}\n")



def main(n: int = 20) -> Dict[str, Dict[str, str]]:
    xlsx_path = INPUT_XLSX
    pairs = load_unique_texts(xlsx_path, n=n)
    passages = build_passages(pairs)
    write_txt(passages, OUTPUT_TXT)

    # Notebook-friendly summary
    print(f"Wrote {len(passages)} passages to {OUTPUT_TXT}")
    for k in list(passages.keys())[:3]:
        print(f"  {k}: id={passages[k]['id']}")
    return passages

In [7]:
# Run in a notebook cell
PASSAGES = main(n=20)

Wrote 20 passages to some_recorded_output\all_passages_dict.txt
  p1: id=anthropology_1_2
  p2: id=anthropology_1_3
  p3: id=anthropology_2_3


### Second, extract information from relevant files and ingest it to QUESTION dictionary. 

In [24]:
from __future__ import annotations
from pathlib import Path
from typing import Dict, List, Any
import pandas as pd
import re
import ast
import random
from pprint import pformat

In [25]:
# ---------- Config ----------
DIR = Path("./some_recorded_output")
PASSAGES_TXT = DIR / "all_passages_dict.txt"       # plain dict literal {...}
BASELINE_XLSX = DIR / "baseline_questions_cleaned_parsed.xlsx"
REQUESTA_XLSX = DIR / "requesta_mcqs_cleaned_parsed.xlsx"
ATTN_XLSX     = DIR / "attention_checker_parsed.xlsx"
OUT_TXT       = DIR / "all_questions_dict.txt"

# Deterministic randomness for attention insertion and sampling
random.seed(42)

LETTER_ID = {"A": "a", "B": "b", "C": "c", "D": "d"}

In [26]:
def to_str(x: Any) -> str:
    if x is None:
        return ""
    s = str(x)
    return s.replace("\r\n", "\n").replace("\r", "\n").replace("\u00A0", " ").strip()

def answer_to_id(ans: Any) -> str | None:
    if ans is None:
        return None
    m = re.search(r"[A-Da-d]", str(ans))
    if not m:
        return None
    return LETTER_ID[m.group(0).upper()]

# ---- Prefix options in DataFrames BEFORE downstream use ----
def _ensure_letter_prefix(s: str, letter: str) -> str:
    s = to_str(s)
    if not s:
        return s
    if re.match(rf"^{letter}\)\s", s):  # already prefixed
        return s
    return f"{letter}) {s}"

def prefix_options_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "optionA" in df.columns:
        df["optionA"] = df["optionA"].map(lambda x: _ensure_letter_prefix(x, "A"))
    if "optionB" in df.columns:
        df["optionB"] = df["optionB"].map(lambda x: _ensure_letter_prefix(x, "B"))
    if "optionC" in df.columns:
        df["optionC"] = df["optionC"].map(lambda x: _ensure_letter_prefix(x, "C"))
    if "optionD" in df.columns:
        df["optionD"] = df["optionD"].map(lambda x: _ensure_letter_prefix(x, "D"))
    return df

def build_choices_from_prefixed(row: Dict[str, Any]) -> List[Dict[str, str]]:
    # DataFrame options already include "A) ...", etc.
    A = to_str(row.get("optionA", ""))
    B = to_str(row.get("optionB", ""))
    C = to_str(row.get("optionC", ""))
    D = to_str(row.get("optionD", ""))
    return [
        {"id": "a", "text": A},
        {"id": "b", "text": B},
        {"id": "c", "text": C},
        {"id": "d", "text": D},
    ]

def build_question(row: Dict[str, Any], answer_col: str) -> Dict[str, Any]:
    # PROMPT now always comes from `question_item`
    return {
        "question_id": to_str(row.get("question_id", "")),
        "prompt": to_str(row.get("question_item", "")),
        "choices": build_choices_from_prefixed(row),
        "correct_choice_id": answer_to_id(row.get(answer_col)),
    }

def insert_attention(qs: List[Dict[str, Any]], attn_df: pd.DataFrame) -> List[Dict[str, Any]]:
    if attn_df.empty:
        return qs
    idx = random.randrange(len(attn_df))
    att_row = attn_df.iloc[idx].to_dict()
    att_q = build_question(att_row, "attention_answer")
    after_n = random.choice([2, 3, 4])
    insert_at = min(after_n, len(qs))  # after Nth means index N (0-based); if list shorter, append
    out = list(qs)
    out.insert(insert_at, att_q)
    return out

def read_passages_dict(path: Path) -> Dict[str, Dict[str, Any]]:
    text = path.read_text(encoding="utf-8")   # PASSAGES is a plain dict literal {...}
    passages = ast.literal_eval(text)
    if not isinstance(passages, dict):
        raise ValueError("PASSAGES text does not contain a dict literal.")
    return passages

def read_excel_or_fail(path: Path, dtype: Dict[str, Any]) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Missing Excel file: {path}")
    return pd.read_excel(path, engine="openpyxl", dtype=dtype)



In [27]:
# ---------- Load inputs ----------
PASSAGES = read_passages_dict(PASSAGES_TXT)

def p_order(k: str) -> int:
    m = re.match(r"p(\d+)$", k)
    return int(m.group(1)) if m else 10_000

ordered_p_keys = sorted(PASSAGES.keys(), key=p_order)

baseline_df = read_excel_or_fail(BASELINE_XLSX, dtype={"textID": str, "question_id": str})
requesta_df = read_excel_or_fail(REQUESTA_XLSX, dtype={"textID": str, "question_id": str})
attn_df     = read_excel_or_fail(ATTN_XLSX,     dtype={"question_id": str})

# Keep only relevant columns (robust to extras)
baseline_cols_keep = [c for c in baseline_df.columns if c in {
    "question_id","textID","baseline_question_type","baseline_question","baseline_answer",
    "question_item","optionA","optionB","optionC","optionD"
}]
requesta_cols_keep = [c for c in requesta_df.columns if c in {
    "question_id","textID","requesta_question_type","requesta_question","requesta_answer",
    "question_item","optionA","optionB","optionC","optionD"
}]
attn_cols_keep = [c for c in attn_df.columns if c in {
    "question_id","attention_question","attention_answer","question_item","optionA","optionB","optionC","optionD"
}]

baseline_df = prefix_options_df(baseline_df[baseline_cols_keep].copy())
requesta_df = prefix_options_df(requesta_df[requesta_cols_keep].copy())
attn_df     = prefix_options_df(attn_df[attn_cols_keep].copy())

# ---------- Build QUESTION dict ----------
QUESTION: Dict[str, Dict[str, Any]] = {}

for pkey in ordered_p_keys:
    text_id = to_str(PASSAGES[pkey].get("id", ""))
    b_rows = baseline_df.loc[baseline_df["textID"] == text_id]
    r_rows = requesta_df.loc[requesta_df["textID"] == text_id]

    baseline_qs: List[Dict[str, Any]] = [
        build_question(row, "baseline_answer")
        for row in b_rows.to_dict(orient="records")
    ]
    requesta_qs: List[Dict[str, Any]] = [
        build_question(row, "requesta_answer")
        for row in r_rows.to_dict(orient="records")
    ]

    # Insert one attention-check into each list (even if empty)
    baseline_qs = insert_attention(baseline_qs, attn_df)
    requesta_qs = insert_attention(requesta_qs, attn_df)

    QUESTION[pkey] = {
        "id": text_id,
        "questions": {
            "baseline": baseline_qs,
            "requesta": requesta_qs,
        },
    }

# ---------- Write output (dict literal only, UTF-8) ----------
OUT_TXT.parent.mkdir(parents=True, exist_ok=True)
OUT_TXT.write_text(pformat(QUESTION, width=100, sort_dicts=False), encoding="utf-8")

print(f"Wrote QUESTION dict → {OUT_TXT}")
print(f"Passages: {len(QUESTION)} | Baseline Qs (incl. attention): {sum(len(v['questions']['baseline']) for v in QUESTION.values())} "
      f"| Requesta Qs (incl. attention): {sum(len(v['questions']['requesta']) for v in QUESTION.values())}")


Wrote QUESTION dict → some_recorded_output\all_questions_dict.txt
Passages: 20 | Baseline Qs (incl. attention): 115 | Requesta Qs (incl. attention): 115
